- using https://database.itreetools.org/#/viableLocations for locations in California that has data in the system
- collect html with https://ourtrees.itreetools.org/#/report? http get method web link
- HTMLs are collect within Los Angeles County only.
- Some data was wrongly input to database and can not find the right location.  It was deleted in LocationsList_2025-05-25.csv

In [1]:
import pandas as pd
import os
from datetime import datetime
from lxml import html
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

if not os.path.exists('data/html_files'):
    os.mkdir('data/html_files')

locationList = pd.read_csv("data/LocationsList_2025-05-25.csv")
zipcode = pd.read_csv("data/zip_code_data.csv")
#print(zipcode.columns)
#print(locationList.columns)
unique_values = locationList['Place'].unique()
print(len(unique_values))
pasadena = locationList[locationList['Place'] == 'Pasadena']
print(pasadena.iloc[0])
cityInCounty = locationList[locationList['County'] == 'Los Angeles']
cityInCounty = cityInCounty.drop_duplicates(subset='Place', keep='first')
print(cityInCounty.head(5))

filename = f'data/html_files/CA_pasadena.html'
Longitude = pasadena.iloc[0]['Longitude']
Latitude = pasadena.iloc[0]['Latitude']
url = f'https://ourtrees.itreetools.org/#/report?longitude={Longitude}&latitude={Latitude}&tab=benefits'
print(url)
# Make a GET request to the URL and save the response content to a file
options = Options()
options.add_argument("headless")  # Don't show browser window


accuTime = 0
print(len(cityInCounty))
for index, row in cityInCounty.iterrows():
    filename = "data/html_files/CA_" + row['Place'] + ".html"

    genHTML = True
    if os.path.exists(filename) :
        if (os.path.getsize(filename)/ 1024) > 50:
            genHTML = False
    
    if genHTML:
        print(filename)
        
        driver = webdriver.Chrome(options=options)
        url = "https://ourtrees.itreetools.org/#/report?longitude="+ str(row['Longitude']) + "&latitude=" + str(row['Latitude']) + "&tab=benefits"
        
        print(url)
        driver.get(url)
        # wait for the page to finish rendering
        time.sleep(30)  # or use WebDriverWait with specific DOM condition
        html = driver.page_source
        accuTime = 0
        while 'Estimating' in html:
            time.sleep(3)
            html = driver.page_source
            accuTime = accuTime + 3
            if accuTime > 480:
                break
        
        with open(filename, 'w') as f:
            f.write(html)
        #print(html)
        driver.quit()
        time.sleep(3)



1682
Nation                United States of America
State                               California
County                             Los Angeles
Place                                 Pasadena
Latitude                             34.176609
Longitude                          -118.172614
Elevation (meters)                  280.573425
Name: 604, dtype: object
                       Nation       State       County                Place  \
506  United States of America  California  Los Angeles                Acton   
507  United States of America  California  Los Angeles         Agoura Hills   
508  United States of America  California  Los Angeles  Agoura Hills-Malibu   
509  United States of America  California  Los Angeles           Agua Dulce   
510  United States of America  California  Los Angeles             Alhambra   

      Latitude   Longitude  Elevation (meters)  
506  34.472777 -118.183696          846.035400  
507  34.153365 -118.761805          282.408356  
508  34.153340 -118

# ourtrees return data example

<p>
  <img src="data/OurTrees_Benefits_Pasadena.png" style="vertical-align: top; width:200px;">
  <img src="data/OurTrees_Community_Pasadena.png" style="vertical-align: top; width:200px;">
  <img src="data/OurTrees_Scorecard_Pasadena.png" style="vertical-align: top; width:200px;">
</p>

In [2]:
from pyquery import PyQuery
import pandas as pd
import re

def getDictItem(pairs, keys):
    if len(pairs) > 1:
        col = items(pairs[0]).text().replace(":", "")
        colstr = col.split(" ")
        if len(colstr) > 3:
            col = " ".join([colstr[0], colstr[1], colstr[2]])
        
        value = items(pairs[1]).text()
        if len(value.split(" ")) > 1:
            col = col + "(" + value.split(" ")[1] + ")"
            value = value.split(" ")[0]
        if "$" in value:
            col = col + "($)"

        value = re.sub(r"[$,]", "", value)

        if col in keys:
            col = col + "-2"
        return {col: value}
    return {}

def getDictItemTR(row):
    tds = list(row("td").items())
    if len(tds) >= 2:
        key = tds[0].text().strip()
        value = tds[1].text().strip()

        if len(value.split(" ")) > 1:
            key = key + "(" + value.split(" ")[1] + ")"
            value = value.split(" ")[0]

        if "$" in value:
            key = key + "($)"
        if "%" in value:
            key = key + "(%)"
        value = re.sub(r"[$,%]", "", value)


        return { key: value}

# get location list with information of html
locationList = pd.read_csv("data/LocationsList_2025-05-25.csv")
cityInCounty = locationList[locationList['County'] == 'Los Angeles']
cityInCounty = cityInCounty.drop_duplicates(subset='Place', keep='first').reset_index()
print(len(cityInCounty))
dfBenefit = pd.DataFrame()
dfCommunity = pd.DataFrame()
dfScore = pd.DataFrame()

# parse html to get data
for index, row in cityInCounty.iterrows():
    filename = "data/html_files/CA_" + row['Place'] + ".html"
    #filename = f'data/html_files/CA_Pasadena.html'    
    print(filename)
    html_str = ""
    with open(filename, 'r',encoding='ISO-8859-1') as f:
        html_str = f.read()

    doc = PyQuery(html_str)
    cityName = re.split(r"[./]", filename)[2]
    row = {"City": cityName}
    # parse benefits page
    labels = [ "City", "percent canopy", "Total Land(acres)", "percent impervious", "impervious(acres)", "Total i-Tree benefits($)", "Carbon Dioxide Uptake($)", "Carbon Sequestered(tn)",
            "CO2 Equivalent1(tn)", "Storm Water Mitigation($)", "Runoff Avoided(MG/yr)", "Rainfall Intercepted(MG/yr)", "Air Pollution Removal($)",
            "Carbon Monoxide(lb/yr)", "Ozone(lb/yr)", "Nitrogen Dioxide(lb/yr)", "Sulfur Dioxide(lb/yr)", "PM2.5(lb/yr)", "Carbon Dioxide Uptake($)-2", "Carbon Storage(tn)", "CO2 Equivalent1(tn)-2"]
    
    # Getting benefits data from benefit section
    benefits = doc("#benefits")
    items = benefits(".justify-content-between")
    for i, item in enumerate(items.items(), 1):
        strong = item.find("strong")
        row = row | getDictItem(strong, row.keys())
        p = item.find("p")
        row = row | getDictItem(p, row.keys())

    topitem = benefits(".my-0")
    toplist1 = items(topitem[0]).text().split(" ")
    toplist2 = items(topitem[1]).text().split(" ")
    #print(toplist1)
    #print(toplist2)
    row["percent canopy"] = re.sub(r"[%]", "", toplist1[0])
    row["Total Land(acres)"] = re.sub(r",", "", toplist1[4])
    row["percent impervious"] = re.sub(r"[%]", "", toplist2[0])
    row["impervious(acres)"] = re.sub(r",", "", toplist2[4])

    #for key, value in row.items():
        #print(f"{key}: {value}")
        
    dfRow = pd.DataFrame([row]) 
    dfRow = dfRow[labels]
    dfBenefit = pd.concat([dfBenefit, dfRow], ignore_index=True)
    #print("benefits data:" + str(len(dfRow.columns)))
    #print(dfRow)

    # Getting community data from community section
    community = doc("#community")
    com_rows = community("tr")
    com_dic =  {"City": cityName}
    for com_rows in com_rows.items():
        com_dic = com_dic | getDictItemTR(com_rows)

    dfCom = pd.DataFrame([com_dic])  
    dfCommunity= pd.concat([dfCommunity, dfCom], ignore_index=True)
    #print("community data:" + str(len(dfCom.columns)))
    #print(dfCom)

    # Getting score data from score section
    score = doc("#scorecard")
    room = score("ul").text().split(" ")
    #print(room)
    score_dic = {"City": cityName, 'Existing canopy(acres)': re.sub(r"[$,%]", "", room[2]), 'Plantable space(acres)': re.sub(r"[$,%]", "", room[5])}
    mt0_paragraphs = score("p.mt-0")
    text1 = mt0_paragraphs.eq(0).text().split(" ")
    text2 = mt0_paragraphs.eq(0).next().text().split(" ")
    #print(text1)
    #print(text2)
    score_dic = score_dic | {'10% benefit($)': re.sub(r"[$,%]", "", text1[13])}
    score_dic = score_dic | {'Carbon beneift(tn)': re.sub(r"[$,%]", "", text2[9])}
    score_dic = score_dic | {'storm water benefit(gal)': re.sub(r"[$,%]", "", text2[13])}
    score_dic = score_dic | {'air benefit(lb)': re.sub(r"[$,%]", "", text2[27])}
    #print(score_dic)
    #dfScore= dfScore.append(pd.DataFrame([score_dic]) , ignore_index=True)
    dfScore= pd.concat([dfScore, pd.DataFrame([score_dic])], ignore_index=True)
    #print("score data:" + str(len(dfscore.columns)))

#print(dfBenefit)
benefitFile = "data/CA_benefit.csv"
dfBenefit.to_csv(benefitFile , index=False)
communitytFile = "data/CA_community.csv"
dfCommunity.to_csv(communitytFile, index=False)
scoreFile = "data/CA_score.csv"
dfScore.to_csv(scoreFile, index=False)

153
data/html_files/CA_Acton.html
data/html_files/CA_Agoura Hills.html
data/html_files/CA_Agoura Hills-Malibu.html
data/html_files/CA_Agua Dulce.html
data/html_files/CA_Alhambra.html
data/html_files/CA_Alondra Park.html
data/html_files/CA_Altadena.html
data/html_files/CA_Arcadia.html
data/html_files/CA_Artesia.html
data/html_files/CA_Avalon.html
data/html_files/CA_Avocado Heights.html
data/html_files/CA_Azusa.html
data/html_files/CA_Baldwin Park.html
data/html_files/CA_Bell.html
data/html_files/CA_Bell Gardens.html
data/html_files/CA_Bellflower.html
data/html_files/CA_Beverly Hills.html
data/html_files/CA_Bradbury.html
data/html_files/CA_Burbank.html
data/html_files/CA_Calabasas.html
data/html_files/CA_Carson.html
data/html_files/CA_Castaic.html
data/html_files/CA_Cerritos.html
data/html_files/CA_Charter Oak.html
data/html_files/CA_Citrus.html
data/html_files/CA_Claremont.html
data/html_files/CA_Commerce.html
data/html_files/CA_Compton.html
data/html_files/CA_Covina.html
data/html_file